In [579]:
import (
	"math"
    "time"
    "fmt"

	"golang.org/x/exp/rand"

	"gonum.org/v1/gonum/stat/distuv"
)

const ThiefCapacity = 3_000


func Max(a, b int64) int64 {
	if a > b {
		return a
	}
	return b
}

func Min(a, b int64) int64 {
	if a < b {
		return a
	}
	return b
}

func MinInt32(a, b int32) int32 {
	if a < b {
		return a
	}
	return b
}

func MaxInt32(a, b int32) int32 {
	if a > b {
		return a
	}
	return b
}


type Heist struct {
	Guards      int32
	Thieves     int32
	TargetCoins int32

	ThiefEvadeBonus      float64
	ThiefCapacityBonus   float64
	GuardEfficiencyBonus float64
	GuardAwarenessBonus float64
}

type HeistOutcome struct {
	Loot              int64
	SuccessfulThieves int32
	CaughtThieves     int32
	SleepingGuards    int32
}

func CalculateHeist(h Heist, rsrc rand.Source) HeistOutcome {
	guards := h.Guards
	thieves := h.Thieves
	guardsf := float64(guards)
	thievesf := float64(thieves)

	dist := distuv.Binomial{
		N:   math.Min(guardsf/3, thievesf),
		P:   0.15 * (1 - h.GuardAwarenessBonus),
		Src: rsrc,
	}
	sleepingGuards := int32(dist.Rand())

	guards = guards - sleepingGuards
	guardsf = float64(guards)
	thievesf = float64(thieves)

	guardsp := guardsf * (h.GuardEfficiencyBonus + 1) * 0.43
	thievesp := thievesf * (h.ThiefEvadeBonus + 1)

	dist = distuv.Binomial{
		N:   thievesf,
		P:   thievesp / (thievesp + guardsp),
		Src: rsrc,
	}
	successfulThieves := int32(dist.Rand())
	caughtThieves := MinInt32(thieves - successfulThieves, guards)
	guardsProtectingLoot := float64(MaxInt32(guards-caughtThieves, 0))
	thiefEfficiency := 0.5 + 0.5/(1+math.Pow(guardsProtectingLoot/12, 0.7))

	maxLoot := int32(float64(successfulThieves) * ThiefCapacity * (h.ThiefCapacityBonus + 1) * thiefEfficiency)
	loot := int64(MinInt32(maxLoot, h.TargetCoins))

	return HeistOutcome{
		Loot:              loot,
		SuccessfulThieves: successfulThieves,
		CaughtThieves:     caughtThieves,
		SleepingGuards:    sleepingGuards,
	}
}



In [586]:
rng := rand.NewSource(uint64(time.Now().UnixNano()))

h := Heist{
    Thieves: 150,
    Guards: 70,
    TargetCoins: 1_000_000,
    ThiefEvadeBonus: 0.75,
    ThiefCapacityBonus: 0.75,
    GuardEfficiencyBonus: 0.9,
    GuardAwarenessBonus: 0.25,
}

r := CalculateHeist(h, rng)

fmt.Printf("%+v\n", r)
fmt.Printf("Profit = %d\n", int32(r.Loot) - r.CaughtThieves * 12000)
fmt.Printf("Overall = %d\n", int32(r.Loot) - r.CaughtThieves * 12000 + r.SleepingGuards * 8000)

rtot := HeistOutcome{}
iterations := 1000
for n := 0; n < iterations; n++ {
    rtmp := CalculateHeist(h, rng)
    rtot.Loot += rtmp.Loot
    rtot.SuccessfulThieves += rtmp.SuccessfulThieves
    rtot.CaughtThieves += rtmp.CaughtThieves
    rtot.SleepingGuards += rtmp.SleepingGuards
}

avgLoot := float64(rtot.Loot) / float64(iterations)
avgSuccessfulThieves := float64(rtot.SuccessfulThieves) / float64(iterations)
avgCaughtThieves := float64(rtot.CaughtThieves) / float64(iterations)
avgSleepingGuards := float64(rtot.SleepingGuards) / float64(iterations)

fmt.Printf("Avg. {Loot:%.3f SuccessfulThieves:%.3f CaughtThieves:%.3f SleepingGuards:%.3f}\n",
           avgLoot, avgSuccessfulThieves, avgCaughtThieves, avgSleepingGuards)
fmt.Printf("Avg. Profit = %.0f\n", avgLoot - avgCaughtThieves * 12000)
fmt.Printf("Avg. Overall = %.0f\n", avgLoot - avgCaughtThieves * 12000 + avgSleepingGuards * 8000)


{Loot:413214 SuccessfulThieves:121 CaughtThieves:29 SleepingGuards:1}
Profit = 65214
Overall = 73214
Avg. {Loot:422067.614 SuccessfulThieves:124.037 CaughtThieves:25.963 SleepingGuards:2.559}
Avg. Profit = 110512
Avg. Overall = 130984


22 <nil>